# 무xx 크롤링

## Import package

In [2]:
import urllib
from selenium import webdriver
import time
import pandas as pd
import sys
import os
from tqdm import tqdm

## file name rule
- {number_ls[k]:02d}_\{page:03d}_\{i:02d}.jpg
- number_ls
    - 01 : 후드 집업
    - 02 : 셔츠/블라우스
    - 03 : 긴팔 티셔츠
    - 04 : 맨투맨/스웨트셔츠
    - 05 : 슈트/블레이저 재킷
    - 06 : 후드 티셔츠
    - 07 : 니트/스웨터
    - 08 : 데님 팬츠
    - 09 : 트레이닝/조거 팬츠
    - 10 : 슈트 팬츠/슬랙스
- page : site page number
- i : image index per page

## page information

In [3]:
number_ls = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
page_count = [27, 147, 101, 245, 25, 153, 77, 69, 112, 49]
page_category = ['002022', '001002', '001010', '001005', '002003', '001004', '001006', '003002', '003004', '003008']

## Crawling Code

In [ ]:
df = pd.DataFrame(columns=['img', 'title', 'price', 'img_url', 'item_url'])
df.index.name = 'ID'
item_url_src = 'https://store.musinsa.com/app/goods'

for k in range(10):
    for page in tqdm(range(1, page_count[k]+1)):
        binary = './chromedriver/chromedriver.exe'
        driver = webdriver.Chrome(binary)

        # 인기탭 창 띄우기
        # 후드 집업
        url = f"https://search.musinsa.com/category/{page_category[k]}?d_cat_cd={page_category[k]}&brand=&rate=&page_kind=search&list_kind=small&sort=pop&sub_sort=&page={page}&display_cnt=90&sale_goods=&ex_soldout=&color=&price1=&price2=&exclusive_yn=&shoeSizeOption=&tags=&campaign_id=&timesale_yn=&q=&includeKeywords=&measure="
        driver.get(url)
        time.sleep(2)

        # 남성 항목 클릭
        btn = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[12]/button[2]')
        btn.click()
        time.sleep(1)
        #pattern
        musinsa = driver.find_elements_by_xpath('//img[@class="lazyload lazy"]')

        for i in range(len(musinsa)):

            img_ele = musinsa[i]
            driver.execute_script("arguments[0].scrollIntoView();", img_ele)
            img = img_ele.get_attribute('src')

            img_file = f"../data/image/mall_image/{number_ls[k]:02d}_{page:03d}_{i:02d}.jpg"
            urllib.request.urlretrieve(img, img_file)
            ele = img_ele.find_element_by_xpath('./../../..')
            name = ele.find_element_by_xpath('./div/p[@class="list_info"]').text
            price = ele.find_element_by_xpath('./div/p[@class="price"]').text
            item_url = f"{item_url_src}/{img.split('/')[6]}"
            try :
                del_price = ele.find_element_by_xpath('./div/p[@class="price"]/del').text
                price = price.replace(del_price,"")
            except :
                pass

            df.loc[len(df)] = [img_file, name, price, img, item_url]

        driver.close()
    df.to_csv(f'../data/csv/mu{number_ls[k]:02d}.csv', encoding='utf-8')
            
if os.path.exists(f'../data/csv/mu_shop.csv') :
    os.remove(f'../data/csv/mu_shop.csv')
else : 
    df.to_csv(f'../data/csv/mu_shop.csv', encoding='utf-8')

 20%|█▉        | 29/147 [11:23<46:03, 23.42s/it] 